In [ ]:
import sys
import numpy as np
import xarray as xr

from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Activation, MaxPool2D, SeparableConv2D, UpSampling2D, concatenate, Conv2DTranspose

from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.utils import plot_model
#from tensorflow import concat
import matplotlib.pyplot as plt

import datetime
%matplotlib inline

# set path to local libraries
dirP_str = '../../../library'
if dirP_str not in sys.path:
    sys.path.append(dirP_str)

import ml_utils as ml

In [ ]:
ds_path='/scr/sci/mhayman/holodec/holodec-ml-data/'

# model_file = "holodec_UNET_16Filt_5Conv_4Pool_mse_linear_random_image_multiplane_data_64x64_5000count300epochs_run1.h5"
# ds_file = 'image_data_256x256_50count.nc'
# ds_file = 'image_data_256x256_5000count.nc'
# ds_file = 'image_data_64x64_5000count.nc'
# ds_file = 'random_image_data_64x64_5000count.nc'
# ds_file = 'random_image_data_64x64_5000count_v02.nc' # 1 um PSF with 1 cm depth
# ds_file = 'random_image_data_64x64_5000count_v03.nc' # 1 um PSF with 10 cm depth
# ds_file = 'random_image_multiplane_data_64x64_5000count.nc' # 1 um PSF with 10 cm depth
# ds_file = "random_image_multiplane_data_64x64_5000count_1particles.nc" # 1 um PSF with 1 cm depth with 1 particles
# ds_file = "random_image_multiplane_data_64x64_5000count_2particles.nc" # 1 um PSF with 1 cm depth with 2 particles


# model_file = "holodec_UNET_16Filt_5Conv_4Pool_mse_linear_random_image_multiplane_data_256x256_5000count_1particles150epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles.nc'

# model_file = "holodec_UNET_16Filt_5Conv_4Pool_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v02150epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v02.nc'

# model_file = "holodec_UNET_16Filt_5Conv_4Pool_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v03_150epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v03.nc'

# model_file = "holodec_UNET_16Filt_5Conv_4Pool_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v04_150epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v04.nc'
# ds_file = 'random_image_multiplane_data_256x256_5000count_3particles_v04.nc'

model_file = "holodec_UNET_16Filt_5Conv_4Pool_5Layers_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v04_150epochs_run1.h5"
ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v04.nc'
# ds_file = 'random_image_multiplane_data_256x256_5000count_3particles_v04.nc'

# model_file = "holodec_UNET_16Filt_7Conv_4Pool_4Layers_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v04_150epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v04.nc'
# ds_file = 'random_image_multiplane_data_256x256_5000count_3particles_v04.nc'

# model_file ="holodec_UNET_16Filt_9Conv_8Pool_3Layers_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v04_51epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v04.nc'

# model_file = "holodec_UNET_16Filt_3Conv_2Pool_8Layers_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v04_151epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v04.nc'

# model_file ="holodec_UNET_16Filt_5Conv_2Pool_6Layers_mse_linear_random_image_multiplane_data_256x256_5000count_1particles_v04_151epochs_run1.h5"
# ds_file = 'random_image_multiplane_data_256x256_5000count_1particles_v04.nc'
# ds_file = 'random_image_multiplane_data_256x256_5000count_3particles_v04.nc'

In [ ]:
ds = xr.open_dataset(ds_path+ds_file)
num_epochs = 0
run_num = 1

In [ ]:
# select holograms to evaluate
index_list = [18,2854,1247,858,3143,832,4021,3921,222,2431,321]
test_all = True  # if this flag is set run all the test cases

In [ ]:
ds.attrs['zmax']

In [ ]:
split_index = np.int(0.7*ds.sizes['hologram_number'])  # number of training+validation points
valid_index = np.int(0.2*ds.sizes['hologram_number'])  # number of validation points
all_labels = ds['labels'].sel(type=['amplitude','z'])

train_labels = all_labels.isel(hologram_number=slice(valid_index,split_index))

if test_all:
    test_labels = all_labels.isel(hologram_number=slice(split_index,None))
else:
    test_labels = all_labels.isel(hologram_number=index_list)

# val_labels = all_labels.isel(hologram_number=slice(None,valid_index))

scaler = ml.MinMaxScalerX(train_labels,dim=('hologram_number','xsize','ysize'))
# scaled_train_labels = scaler.fit_transform(train_labels)
# scaled_val_labels = scaler.fit_transform(val_labels)
scaled_test_labels = scaler.fit_transform(test_labels)
# scaled_all_labels = scaler.fit_transform(all_labels)

In [ ]:
if test_all:
    in_data = ds['image'].isel(hologram_number=slice(split_index,None))
else:
    in_data = ds['image'].isel(hologram_number=index_list)

In [ ]:
if not 'channel' in in_data.dims:
    in_data = in_data.expand_dims("channel", 3)

In [ ]:
scaled_in_data = in_data

In [ ]:
# load the CNN model
mod = load_model(ds_path+"/models/"+model_file)

In [ ]:
cnn_start = datetime.datetime.now()
preds_out = mod.predict(scaled_in_data.values, batch_size=64)
cnn_stop = datetime.datetime.now()
print(f"{scaled_in_data.values.shape[0]} samples in {(cnn_stop-cnn_start).total_seconds()} seconds")
print(f"for {(cnn_stop-cnn_start).total_seconds()/scaled_in_data.values.shape[0]} seconds per hologram")

In [ ]:
preds_out.shape

In [ ]:
preds_out_da = xr.DataArray(preds_out,dims=('hologram_number','xsize','ysize','type'),
                            coords=all_labels.coords)

In [ ]:
preds_original = scaler.inverse_transform(preds_out_da)

In [ ]:
test_labels.shape

In [ ]:
# evaluate each predicted particle location by
# the prediction with the least error
# the prediction with the highest amplitude term
z_min = []
z_amp = []
z_act = []
a_min = []
a_amp = []
a_act = []
for ih in test_labels.coords['hologram_number']:
    preds0 = preds_original.sel(hologram_number=ih)
    ampset = np.nonzero(test_labels.sel(hologram_number=ih,type='amplitude').values > 0.1)
    zset = np.unique(test_labels.sel(hologram_number=ih,type='z').values[ampset])
    for iz,z in enumerate(zset):
        ipart = np.nonzero(test_labels.sel(hologram_number=ih,type='z').values==z)
        zpred = preds0.sel(type='z').values[ipart]
        apred = preds0.sel(type='amplitude').values[ipart]
        iamp = np.argmax(apred)
        imin = np.argmin(np.abs(zpred-z))
        z_act += [z]
        z_amp += [zpred[iamp]]
        z_min += [zpred[imin]]
        a_act += [np.max(apred)]
        a_amp += [apred[iamp]]
        a_min += [apred[imin]]
z_act = np.array(z_act)
z_amp = np.array(z_amp)
z_min = np.array(z_min)
a_act = np.array(a_act)
a_amp = np.array(a_amp)
a_min = np.array(a_min)
        

In [ ]:
print(z_min[0:5])
print(z_act[0:5])

In [ ]:
z_one_to_one = [z_act.min()*1e3,z_act.max()*1e3]
fig, ax = plt.subplots(1,1, figsize=(4, 4))
ax.plot(z_one_to_one,z_one_to_one,'k--')
ax.scatter(z_act*1e3,z_amp*1e3,s=5,alpha=0.5,label='max amplitude')
ax.scatter(z_act*1e3,z_min*1e3,s=5,alpha=0.5,label='min error')
ax.set_xlabel('z actual [mm]')
ax.set_ylabel('z predicted [mm]')
ax.grid(b=True)
ax.legend()
plt.savefig("results/"+model_file.replace(".h5","")+f"_Zscatter"+f"_{num_epochs}epochs_run{run_num}_"+ds_file.replace(".nc","")+".png",dpi=300)

In [ ]:
hbins = np.linspace(-10,10,100)
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].hist(z_act*1e3-z_amp*1e3,bins=hbins)
ax[0].set_xlabel('z error [mm]')
ax[0].set_title('Max Amplitude')
ax[0].minorticks_on()
ax[0].grid(b=True)
ax[0].text(0.02, 0.98, 'RMSE = %.02f mm'%np.sqrt(np.mean((z_act*1e3-z_amp*1e3)**2)), 
     horizontalalignment="left",verticalalignment="top", transform=ax[0].transAxes)

ax[1].hist(z_act*1e3-z_min*1e3,bins=hbins)
ax[1].set_xlabel('z error [mm]')
ax[1].set_title('Min Error')
ax[1].minorticks_on()
ax[1].grid(b=True)
ax[1].text(0.02, 0.98, 'RMSE = %.02f mm'%np.sqrt(np.mean((z_act*1e3-z_min*1e3)**2)), 
     horizontalalignment="left",verticalalignment="top", transform=ax[1].transAxes)
plt.savefig("results/"+model_file.replace(".h5","")+f"_Zhistogram"+f"_{num_epochs}epochs_run{run_num}_"+ds_file.replace(".nc","")+".png",dpi=300)

In [ ]:
iscatter = np.nonzero(preds_original.sel(type='amplitude').values.flatten() > 0.2)
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
for a, clabel in enumerate(all_labels.coords['type'].values):
    ax=axes.ravel()[a]
    ax.scatter(test_labels.sel(type=clabel).values.flatten()[iscatter], preds_original.sel(type=clabel).values.flatten()[iscatter], 1, 'k')
    diag = np.linspace(test_labels.sel(type=clabel).min(), test_labels.sel(type=clabel).max(), 10)
    ax.plot(diag, diag, 'b--' )
    ax.set_title(clabel)
# plt.savefig("results/"+model_file.replace(".h5","")+f"_SampleScatterPlot"+f"_{num_epochs}epochs_run{run_num}_"+ds_file.replace(".nc","")+".png",dpi=300)

In [ ]:
diff_cmap = plt.get_cmap('seismic')
diff_cmap.set_bad(color='gray')

z_cmap = plt.get_cmap('viridis')
z_cmap.set_bad(color='gray')

diff_res = ds.attrs['zmax']*0.1

for ind in range(len(index_list)):
    fig, ax = plt.subplots(2, 3, figsize=(12, 8))
    ax = ax.ravel()
    
    inan_mask = np.nonzero((preds_original.sel(type='amplitude',hologram_number=ind).values < 0.1)* \
        (test_labels.sel(type='amplitude',hologram_number=ind).values < 0.1))
    nan_mask = np.ones(preds_original.sel(type='amplitude',hologram_number=ind).values.shape)
    nan_mask[inan_mask] = np.nan
    
    ax[0].imshow(preds_original.sel(type='amplitude',hologram_number=ind).values,vmin=0,vmax=1)
    ax[1].imshow(test_labels.sel(type='amplitude',hologram_number=ind).values,vmin=0,vmax=1)
    ax[2].imshow((preds_original.sel(type='amplitude',hologram_number=ind).values-test_labels.sel(type='amplitude',hologram_number=ind).values)*nan_mask,vmin=-1,vmax=1,cmap=diff_cmap)
    # ax[2].imshow(scaled_in_data.values[ind,:,:,0])
    ax[3].imshow(preds_original.sel(type='z',hologram_number=ind).values*nan_mask,vmin=0,vmax=ds.attrs['zmax'],cmap=z_cmap)
    ax[4].imshow(test_labels.sel(type='z',hologram_number=ind).values*nan_mask,vmin=0,vmax=ds.attrs['zmax'],cmap=z_cmap)
    ax[5].imshow((preds_original.sel(type='z',hologram_number=ind).values-test_labels.sel(type='z',hologram_number=ind).values)*nan_mask,vmin=-diff_res,vmax=diff_res,cmap=diff_cmap)
    plt.savefig("results/"+model_file.replace(".h5","")+f"_ExampleImage{index_list[ind]}"+f"_{num_epochs}epochs_run{run_num}_"+ds_file.replace(".nc","")+".png",dpi=300)

In [ ]:
channel_number = in_data.sizes['channel']
# index_list = [18]
for ind in range(len(index_list)):
    fig, ax = plt.subplots(2, channel_number//2, figsize=(channel_number*3, 8))
    for ai in range(channel_number):
        axind = ai//2+np.mod(ai,2)*channel_number//2
        ax[np.mod(ai,2),ai//2].imshow(scaled_in_data.isel(channel=ai,hologram_number=ind),vmin=-0.25,vmax=0.25)
    plt.savefig("results/"+model_file.replace(".h5","")+f"_ExampleInput{index_list[ind]}"+f"_{num_epochs}epochs_run{run_num}_"+ds_file.replace(".nc","")+".png",dpi=300)

In [ ]:
in_data.sizes

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
xg,yg = np.meshgrid(preds_original['xsize'].values,preds_original['ysize'].values)

In [ ]:
for ind,indact in enumerate(index_list):
    ipart = np.nonzero(preds_original.sel(type='amplitude',hologram_number=ind).values > 0.2)
    ipart_label = np.nonzero(test_labels.sel(type='amplitude',hologram_number=ind).values > 0.2)
    amp_p = preds_original.sel(type='amplitude',hologram_number=ind).values[ipart]
    z_p = preds_original.sel(type='z',hologram_number=ind).values[ipart]
    x_p = xg[ipart]
    y_p = yg[ipart]
    
    z_l = test_labels.sel(type='z',hologram_number=ind).values[ipart_label]
    x_l = xg[ipart_label]
    y_l = yg[ipart_label]

    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(z_p,x_p,y_p,c=amp_p,vmin=0,vmax=1,s=1)
    ax.scatter(z_l,x_l,y_l,c='k',s=1)
    ax.set_xlim([ds.attrs['zmin'],ds.attrs['zmax']])
    ax.set_ylim([preds_original['xsize'].values[0],preds_original['xsize'].values[-1]])
    ax.set_zlim([preds_original['ysize'].values[0],preds_original['ysize'].values[-1]])
    ax.set_xlabel('z')
    ax.set_ylabel('x')
    ax.set_zlabel('y')
    plt.savefig("results/"+model_file.replace(".h5","")+f"_Scatter3D{index_list[ind]}"+f"_{num_epochs}epochs_run{run_num}_"+ds_file.replace(".nc","")+".png",dpi=300)

In [ ]:
preds_original['xsize'].values[0]